In [2]:
import gensim
import os
import collections
import smart_open
import random

In [3]:
ls -lh ../paraphrases/

total 8,3M
-rw-r--r-- 1 fogside fogside 4,8M ноя 28  2015 corpus.xml
-rw-r--r-- 1 fogside fogside 1,2K апр 16  2015 LICENSE
-rw-r--r-- 1 fogside fogside 3,6M ноя 28  2015 paraphrases.xml


In [45]:
import xml.etree.ElementTree as ET
parser = ET.XMLParser(encoding="utf-8")
tree = ET.parse('../paraphrases/paraphrases.xml', parser=parser)
for elem in tree.iter(tag='value'):
    print(elem.tag, elem.attrib, elem.text)

value {'name': 'id'} 1
value {'name': 'id_1'} 201
value {'name': 'id_2'} 8159
value {'name': 'text_1'} Полицейским разрешат стрелять на поражение по гражданам с травматикой.
value {'name': 'text_2'} Полиции могут разрешить стрелять по хулиганам с травматикой.
value {'name': 'jaccard'} 0.65
value {'name': 'class'} 0
value {'name': 'id'} 2
value {'name': 'id_1'} 202
value {'name': 'id_2'} 8158
value {'name': 'text_1'} Право полицейских на проникновение в жилище решили ограничить.
value {'name': 'text_2'} Правила внесудебного проникновения полицейских в жилище уточнят.
value {'name': 'jaccard'} 0.5
value {'name': 'class'} 0
value {'name': 'id'} 3
value {'name': 'id_1'} 273
value {'name': 'id_2'} 8167
value {'name': 'text_1'} Президент Египта ввел чрезвычайное положение в мятежных городах.
value {'name': 'text_2'} Власти Египта угрожают ввести в стране чрезвычайное положение.
value {'name': 'jaccard'} 0.611429
value {'name': 'class'} 0
value {'name': 'id'} 4
value {'name': 'id_1'} 220
valu

value {'name': 'id_1'} 151
value {'name': 'id_2'} 4258
value {'name': 'text_1'} Минфин не поддерживает идею создания ведомства по делам сирот.
value {'name': 'text_2'} В Госдуме предложили создать ведомство по делам сирот.
value {'name': 'jaccard'} 0.552186
value {'name': 'class'} -1
value {'name': 'id'} 179
value {'name': 'id_1'} 64
value {'name': 'id_2'} 4177
value {'name': 'text_1'} СМИ: Взрыв в Тель-Авиве - попытка ликвидации криминального авторитета.
value {'name': 'text_2'} Взрыв в Тель-Авиве был покушением на криминального "авторитета".
value {'name': 'jaccard'} 0.632644
value {'name': 'class'} 1
value {'name': 'id'} 180
value {'name': 'id_1'} 155
value {'name': 'id_2'} 4258
value {'name': 'text_1'} В России могут создать министерство по делам сирот.
value {'name': 'text_2'} В Госдуме предложили создать ведомство по делам сирот.
value {'name': 'jaccard'} 0.5
value {'name': 'class'} 1
value {'name': 'id'} 181
value {'name': 'id_1'} 283
value {'name': 'id_2'} 4393
value {'name': '

value {'name': 'text_2'} Ущерб от челябинского метеорита превысит миллиард рублей.
value {'name': 'jaccard'} 0.771517
value {'name': 'class'} 1
value {'name': 'id'} 423
value {'name': 'id_1'} 481
value {'name': 'id_2'} 4598
value {'name': 'text_1'} Закон о "резиновых квартирах" принят депутатами в первом чтении.
value {'name': 'text_2'} Госдума приняла в первом чтении закон о «резиновых квартирах».
value {'name': 'jaccard'} 0.857143
value {'name': 'class'} 1
value {'name': 'id'} 424
value {'name': 'id_1'} 580
value {'name': 'id_2'} 4681
value {'name': 'text_1'} По сирийскому Алеппо нанесли ракетный удар: десятки погибших.
value {'name': 'text_2'} По Алеппо нанесли новый ракетный удар.
value {'name': 'jaccard'} 0.693026
value {'name': 'class'} 0
value {'name': 'id'} 425
value {'name': 'id_1'} 581
value {'name': 'id_2'} 4682
value {'name': 'text_1'} В США обнаружена утечка из 6 резервуаров для ядерных отходов.
value {'name': 'text_2'} В США произошла утечка в хранилище ядерных отходов.
v

value {'name': 'id'} 662
value {'name': 'id_1'} 566
value {'name': 'id_2'} 4658
value {'name': 'text_1'} В деле о землях Минобороны "засветилась" супруга чиновника Ленобласти.
value {'name': 'text_2'} Жену чиновника из Ленобласти заподозрили в махинациях с землями Минобороны.
value {'name': 'jaccard'} 0.664286
value {'name': 'class'} 0
value {'name': 'id'} 663
value {'name': 'id_1'} 480
value {'name': 'id_2'} 4571
value {'name': 'text_1'} Крушение самолета Ан-24 в Донецке.
value {'name': 'text_2'} Названа предварительная версия крушения Ан-24 под Донецком.
value {'name': 'jaccard'} 0.535826
value {'name': 'class'} 0
value {'name': 'id'} 664
value {'name': 'id_1'} 381
value {'name': 'id_2'} 4484
value {'name': 'text_1'} Самый дорогой билет на Олимпиаду в Сочи стоит 50 тыс. руб.
value {'name': 'text_2'} Оргкомитет «Сочи-2014» заработает на билетах 250 миллионов долларов.
value {'name': 'jaccard'} 0.264575
value {'name': 'class'} -1
value {'name': 'id'} 665
value {'name': 'id_1'} 558
valu

value {'name': 'id'} 916
value {'name': 'id_1'} 757
value {'name': 'id_2'} 4858
value {'name': 'text_1'} В Ватикане начались выборы нового Папы Римского.
value {'name': 'text_2'} В Ватикане начались выборы нового папы.
value {'name': 'jaccard'} 0.931128
value {'name': 'class'} 1
value {'name': 'id'} 917
value {'name': 'id_1'} 691
value {'name': 'id_2'} 4784
value {'name': 'text_1'} Задержан подозреваемый в покушении на балетмейстера С.Филина.
value {'name': 'text_2'} Задержан подозреваемый в покушении на худрука балета Большого театра.
value {'name': 'jaccard'} 0.507093
value {'name': 'class'} 0
value {'name': 'id'} 918
value {'name': 'id_1'} 690
value {'name': 'id_2'} 4785
value {'name': 'text_1'} "Коммерсантъ" покидает директор информационного центра М.Лукин.
value {'name': 'text_2'} Из «Коммерсанта» ушел директор информационного центра.
value {'name': 'jaccard'} 0.547723
value {'name': 'class'} 0
value {'name': 'id'} 919
value {'name': 'id_1'} 749
value {'name': 'id_2'} 4859
value {

...

value {'name': 'id_2'} 32799
value {'name': 'text_1'} FT: ЕС недоволен тем, что Киев медлит с выполнением мирных соглашений
value {'name': 'text_2'} FT сообщила о намерениях ЕС убедить Киев выполнить Минские соглашения
value {'name': 'jaccard'} 0.5555555555555556
value {'name': 'class'} -1
value {'name': 'id'} 24170
value {'name': 'id_1'} 32800
value {'name': 'id_2'} 32801
value {'name': 'text_1'} Фонд Клинтон признал ошибки после критики за получение пожертвований
value {'name': 'text_2'} Фонд Клинтонов признал ошибки при получении пожертвований
value {'name': 'jaccard'} 0.8555263514564874
value {'name': 'class'} 0
value {'name': 'id'} 24171
value {'name': 'id_1'} 32802
value {'name': 'id_2'} 32803
value {'name': 'text_1'} Байкеры из России начали пересекать границу с Польшей, пишут СМИ
value {'name': 'text_2'} Несколько байкеров из России поодиночке пересекли границу с Польшей
value {'name': 'jaccard'} 0.7000372013924213
value {'name': 'class'} 0
value {'name': 'id'} 24172
value {'na

value {'name': 'text_2'} США предложили России продолжить ядерное разоружение
value {'name': 'jaccard'} 0.4626813958590448
value {'name': 'class'} -1
value {'name': 'id'} 24476
value {'name': 'id_1'} 33173
value {'name': 'id_2'} 33174
value {'name': 'text_1'} Глава Крыма требует разобраться с воровством на объектах Коломойского
value {'name': 'text_2'} Аксенов потребовал разобраться с воровством на объектах Коломойского
value {'name': 'jaccard'} 0.6480740698407861
value {'name': 'class'} 1
value {'name': 'id'} 24477
value {'name': 'id_1'} 33175
value {'name': 'id_2'} 33176
value {'name': 'text_1'} Roshen сообщает об аресте имущества липецкой фабрики на 2 млрд рублей
value {'name': 'text_2'} Басманный суд арестовал имущество липецкой фабрики Roshen
value {'name': 'jaccard'} 0.5291502622129182
value {'name': 'class'} -1
value {'name': 'id'} 24478
value {'name': 'id_1'} 33177
value {'name': 'id_2'} 33178
value {'name': 'text_1'} В зоне отчуждения Чернобыльской АЭС загорелся лес
value {'na

value {'name': 'class'} 1
value {'name': 'id'} 24708
value {'name': 'id_1'} 33496
value {'name': 'id_2'} 33495
value {'name': 'text_1'} Радиационный фон в районе пожара около ЧАЭС ниже обычно уровня
value {'name': 'text_2'} Роспотребнадор: Радиационный фон в РФ после пожара в зоне ЧАЭС в норме
value {'name': 'jaccard'} 0.6481085116376291
value {'name': 'class'} 0
value {'name': 'id'} 24709
value {'name': 'id_1'} 33497
value {'name': 'id_2'} 33498
value {'name': 'text_1'} Кадыров готов дать показания в рамках дела об убийстве Немцова
value {'name': 'text_2'} Кадыров заявил о готовности дать показания по делу об убийстве Немцова
value {'name': 'jaccard'} 0.8535714285714283
value {'name': 'class'} 1
value {'name': 'id'} 24710
value {'name': 'id_1'} 33421
value {'name': 'id_2'} 33499
value {'name': 'text_1'} Падение аппарата "Мессенджер" создаст на Меркурии 16-метровый кратер
value {'name': 'text_2'} НАСА создаст на поверхности Меркурия 16-метровый кратер
value {'name': 'jaccard'} 0.678934

value {'name': 'class'} -1
value {'name': 'id'} 24996
value {'name': 'id_1'} 33927
value {'name': 'id_2'} 33926
value {'name': 'text_1'} Милиционеры погибли в Киеве при задержании подозреваемых в краже $38
value {'name': 'text_2'} Трое стрелявших в милиционеров в Киеве задержаны
value {'name': 'jaccard'} 0.45074893585520875
value {'name': 'class'} -1
value {'name': 'id'} 24997
value {'name': 'id_1'} 33928
value {'name': 'id_2'} 33929
value {'name': 'text_1'} ИГ взяла ответственность за стрельбу на выставке карикатур в США
value {'name': 'text_2'} СМИ: "ИГ" взяла ответственность за стрельбу на выставке карикатур в США
value {'name': 'jaccard'} 0.9621404708847278
value {'name': 'class'} 1
value {'name': 'id'} 24998
value {'name': 'id_1'} 33930
value {'name': 'id_2'} 33931
value {'name': 'text_1'} Спасатели обнаружили тела ста человек, погибших из-за лавины в Непале
value {'name': 'text_2'} Обнаружены тела еще ста человек, погибших из-за лавины в Непале
value {'name': 'jaccard'} 0.9621404

value {'name': 'id_2'} 34308
value {'name': 'text_1'} Нетаньяху объявил, что сформировал новую правящую в Израиле коалицию
value {'name': 'text_2'} Нетаньяху сформировал новую правящую коалицию
value {'name': 'jaccard'} 0.7678830085819092
value {'name': 'class'} 1
value {'name': 'id'} 25259
value {'name': 'id_1'} 34309
value {'name': 'id_2'} 34310
value {'name': 'text_1'} Самолет с эвакуированными из Непала гражданами Украины вернулся в Киев
value {'name': 'text_2'} Ил-76 с вывезенными из Непала гражданами Украины долетел до Киева
value {'name': 'jaccard'} 0.657142857142857
value {'name': 'class'} 0
value {'name': 'id'} 25260
value {'name': 'id_1'} 34311
value {'name': 'id_2'} 34310
value {'name': 'text_1'} МО Украины: самолет с эвакуированными из Непала ждут в Киеве ночью
value {'name': 'text_2'} Ил-76 с вывезенными из Непала гражданами Украины долетел до Киева
value {'name': 'jaccard'} 0.48107023544236377
value {'name': 'class'} 0
value {'name': 'id'} 25261
value {'name': 'id_1'} 343

In [ ]:
data = 